In [2]:
import pandas as pd
from os import listdir
import numpy as np

In [3]:
directory = "ados_datasets/"

In [4]:
filename='current_data.txt'
txtname = directory+filename
data = pd.read_csv(txtname, sep="\t")

In [27]:
potentiallyUseful = ['ados diagnosis classification',
        'age in months at the time of the interview/test/sampling/imaging.',
        'anxiety',
        'hand and finger and other complex mannerisms',
        'imagination/creativity', 
        'immediate echolalia',
        'quality of social overtures', 
        'self-injurious behavior',
        'isMale', 
        'shared enjoyment in interaction',
        'tantrums, aggression, negative or disruptive behavior',
        'unusual eye contact',
        'diagnosis']

In [6]:
print ('Number of rows with at least 1 NaN features')
print (data[data.isnull().sum(axis=1) > 0].shape[0])
# Remove them for now
data = data[data.isnull().sum(axis=1) == 0]
print (str(data.shape[0]) + ' rows left')

Number of rows with at least 1 NaN features
1339
2733 rows left


In [7]:
# remove the rows with unknown (9.) variables
# and make the 8. (not applicable) 0 (typical)
# and make the 3. (severe) 2 (not that severe)
for col in potentiallyUseful[2:]:
    data = data[data[col] != 9]
    data[col] = data[col].apply(lambda x: 0 if x == 8 else x)
    data[col] = data[col].apply(lambda x: 2 if x == 3 else x)    

In [8]:
for col in data[potentiallyUseful]:
    print("COLUMN:", col)
    print(data[col].value_counts())

COLUMN: ados diagnosis classification
ASD                      478
non-spectrum             442
2                        376
none                     290
0                        219
Autism                   118
autism                   102
non                       91
Non-spectrum              89
Unknown                   84
Non-Spectrum              69
aut                       62
autism spectrum           58
Nonspectrum               52
Autism Spectrum           38
9                         25
1                         25
non spectrum              23
autism spect              14
typical                   11
Non spectrum               6
nonspectrum                6
Typical                    6
Non Spectrum               4
Nonspectrum.               4
3                          4
nonspectrum/typical        3
Non-Spec                   3
non-spec                   3
Autism spectrum            3
Autism Spectrum.           2
nonspec                    2
Autism Spectrum?           2
Autis

In [28]:
data[potentiallyUseful].to_csv('ados_datasets/finalDataV1.csv', sep=';', index=False)

In [11]:
data['diagnosis'] = data['diagnosis'].apply(lambda x: 1 if x == 2 else x) 
data= data[data.diagnosis >= 0]

In [26]:
data['isMale'] = data['sex of the subject'].apply(lambda x: 1 if x == 'M' else 0) 

In [46]:
data[potentiallyUseful].corr()

,age in months at the time of the interview/test/sampling/imaging.,anxiety,hand and finger and other complex mannerisms,imagination/creativity,immediate echolalia,quality of social overtures,self-injurious behavior,isMale,shared enjoyment in interaction,"tantrums, aggression, negative or disruptive behavior",unusual eye contact,diagnosis
age in months at the time of the interview/test/sampling/imaging.,1.000000,-0.090634,0.169771,0.039551,0.014768,0.236480,0.140257,0.066696,0.233195,-0.074960,0.251066,-0.043004
anxiety,-0.090634,1.000000,0.002060,0.105404,0.071787,0.030005,-0.003020,-0.055797,0.049894,0.062059,-0.008694,0.066537
hand and finger and other complex mannerisms,0.169771,0.002060,1.000000,0.396220,0.179522,0.544921,0.190004,0.101577,0.383812,0.119530,0.488809,0.286440
imagination/creativity,0.039551,0.105404,0.396220,1.000000,0.117380,0.531777,0.148770,0.160758,0.465312,0.206934,0.433428,0.272630
immediate echolalia,0.014768,0.071787,0.179522,0.117380,1.000000,0.236666,-0.000703,0.049982,0.128130,0.066119,0.258457,0.272843
quality of social overtures,0.236480,0.030005,0.544921,0.531777,0.236666,1.000000,0.209271,0.168133,0.623203,0.256780,0.709134,0.532987
self-injurious behavior,0.140257,-0.003020,0.190004,0.148770,-0.000703,0.209271,1.000000,0.062170,0.163843,0.149724,0.177077,0.065280
isMale,0.066696,-0.055797,0.101577,0.160758,0.049982,0.168133,0.062170,1.000000,0.131159,0.058802,0.150613,0.116500
shared enjoyment in interaction,0.233195,0.049894,0.383812,0.465312,0.128130,0.623203,0.163843,0.131159,1.000000,0.228244,0.524898,0.331309
"tantrums, aggression, negative or disruptive behavior",-0.074960,0.062059,0.119530,0.206934,0.066119,0.256780,0.149724,0.058802,0.228244,1.000000,0.168595,0.181847


In [71]:
notCorrelated = ['self-injurious behavior']
notX = ['diagnosis', 'ados diagnosis classification']

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np

In [73]:
X = data[potentiallyUseful].drop(notX, axis=1).values
y = data['diagnosis']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [75]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
Z = logreg.predict(X_test)



In [76]:
f1_score(Z, y_test, average='macro'), f1_score(Z, y_test, average='micro'), f1_score(Z, y_test, average='weighted')


(0.796802324475833, 0.8090737240075614, 0.8093569101506013)

In [70]:
f1_score(Z, y_test, average='macro'), f1_score(Z, y_test, average='micro'), f1_score(Z, y_test, average='weighted')


(0.7924133420536299, 0.8128544423440454, 0.8122387465521654)

In [40]:
f1_score(Z, y_test, average='macro')

0.3383742911153119